# 액트버스 행동 분석하기

## 1. 준비하기

In [ ]:
! pip install actverse@git+https://github.com/actnova-inc/actverse-analysis

## 2. 시작하기

In [ ]:
from actverse.utils import load_json

json_path = input("Downloadable url or local file path:")

prediction = load_json(json_path)

In [ ]:
from actverse.utils.notebook import display_body_parts_checkbox

checkboxes = display_body_parts_checkbox(
    description="분석을 원하는 부위를 선택해주세요.", lang="ko"
)

## 3. 행동 분석하기

In [ ]:
from actverse.analysis import measure_physical_metrics
from actverse.utils.notebook import get_checked

body_parts = get_checked(checkboxes)
body_part_names = [str(body_part) for body_part in body_parts]
metrics, ids = measure_physical_metrics(prediction, body_parts)

## 4. 결과 보기

이동 지표

In [ ]:
img_width = prediction["metadata"]["origin_width"]
img_height = prediction["metadata"]["origin_height"]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from actverse.utils import moving_average

mouse_id = ids[0]

unit_width = 400
unit_height = 350

# first plot
fig1 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)
for i, body_part in enumerate(body_parts):
    x = metrics[mouse_id][body_part]["timestamp"]
    y_1 = metrics[mouse_id][body_part]["cumulative_distance_change"]
    go_1 = go.Scatter(x=x, y=y_1, mode="lines", name=str(body_part))
    fig1.add_trace(go_1, row=1, col=i + 1)
    fig1.update_xaxes(title_text="시간 (초)", row=1, col=i + 1)
    fig1.update_yaxes(title_text="거리 (픽셀)", row=1, col=i + 1)
fig1.update_layout(
    title="이동 거리", height=unit_height, width=unit_width * len(body_parts)
)
fig1.show()

# second plot
fig2 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)
for i, body_part in enumerate(body_parts):
    x = metrics[mouse_id][body_part]["timestamp"]
    position = metrics[mouse_id][body_part]["speed"]
    go_2 = go.Scatter(x=x, y=position, mode="lines", name=str(body_part))
    fig2.add_trace(go_2, row=1, col=i + 1)
    fig2.update_xaxes(title_text="시간 (초)", row=1, col=i + 1)
    fig2.update_yaxes(title_text="속력 (픽셀/초)", row=1, col=i + 1)
fig2.update_layout(
    title="속력", height=unit_height, width=unit_width * len(body_parts)
)
fig2.show()

# third subplot
fig3 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)
for i, body_part in enumerate(body_parts):
    x = metrics[mouse_id][body_part]["timestamp"]
    y_3 = moving_average(metrics[mouse_id][body_part]["speed"], 25)
    go_3 = go.Scatter(x=x, y=y_3, mode="lines", name=str(body_part))
    fig3.add_trace(go_3, row=1, col=i + 1)
    fig3.update_xaxes(title_text="시간 (초)", row=1, col=i + 1)
    fig3.update_yaxes(title_text="속력 (픽셀/초)", row=1, col=i + 1)
fig3.update_layout(
    title="속력 (단순화)", height=unit_height, width=unit_width * len(body_parts)
)
fig3.show()

# fourth subplot
fig4 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)
for i, body_part in enumerate(body_parts):
    x = metrics[mouse_id][body_part]["timestamp"]
    y_4 = metrics[mouse_id][body_part]["speed"]
    go_4 = go.Histogram(x=y_4, xbins=dict(size=1), name=str(body_part))
    fig4.add_trace(go_4, row=1, col=i + 1)
    fig4.update_xaxes(title_text="속력 (픽셀/초)", row=1, col=i + 1)
    fig4.update_yaxes(title_text="빈도", row=1, col=i + 1)
fig4.update_layout(
    title="속력 (히스토그램)", height=unit_height, width=unit_width * len(body_parts)
)
fig4.show()

#  fifth subplot
fig5 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)
for i, body_part in enumerate(body_parts):
    x = metrics[mouse_id][body_part]["timestamp"]
    y_5 = metrics[mouse_id][body_part]["average_speed"]
    go_5 = go.Scatter(x=x, y=y_5, mode="lines", name=str(body_part))
    fig5.add_trace(go_5, row=1, col=i + 1)
    fig5.update_xaxes(title_text="시간 (초)", row=1, col=i + 1)
    fig5.update_yaxes(title_text="속력 (픽셀/초)", row=1, col=i + 1)
fig5.update_layout(
    title="평균 속력", height=unit_height, width=unit_width * len(body_parts)
)
fig5.show()

방향 지표

In [ ]:
mouse_id = ids[0]

# subplot for ploty
n_rows, n_cols = 3, 2
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[
        "몸 각도",
        "각속도",
        "각속도 (단순화)",
        "몸 각도 (히스토그램)",
        "각속도 (히스토그램)",
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
)

x = metrics[mouse_id]["body center"]["timestamp"]

# first subplot
y_1 = metrics[mouse_id]["body center"]["angle"]
go_1 = go.Scatter(x=x, y=y_1, mode="lines", name="몸 각도")
fig.add_trace(go_1, row=1, col=1)
fig.update_xaxes(title_text="시간 (초)", row=1, col=1)
fig.update_yaxes(title_text="각도 (도)", row=1, col=1)

# second subplot
position = metrics[mouse_id]["body center"]["angular_speed"]
go_2 = go.Scatter(x=x, y=position, mode="lines", name="각속도")
fig.add_trace(go_2, row=1, col=2)
fig.update_xaxes(title_text="시간 (초)", row=1, col=2)
fig.update_yaxes(title_text="각속도 (도/초)", row=1, col=2)

# third subplot
y_3 = moving_average(metrics[mouse_id]["body center"]["angular_speed"], 25)
go_3 = go.Scatter(x=x, y=y_3, mode="lines", name="각속도 (단순화)")
fig.add_trace(go_3, row=2, col=1)
fig.update_xaxes(title_text="시간 (초)", row=2, col=1)
fig.update_yaxes(title_text="각속도 (도/초)", row=2, col=1)

# fourth subplot
y_4 = metrics[mouse_id]["body center"]["angle"]
go_4 = go.Histogram(x=y_4, xbins=dict(size=1), name="몸 각도 (히스토그램)")
fig.add_trace(go_4, row=2, col=2)
fig.update_xaxes(title_text="각도 (도)", row=2, col=2)
fig.update_yaxes(title_text="빈도", row=2, col=2)

# fifth subplot
y_5 = metrics[mouse_id]["body center"]["angular_speed"]
go_5 = go.Histogram(x=y_5, xbins=dict(size=1), name="각속도 (히스토그램)")
fig.add_trace(go_5, row=3, col=1)
fig.update_xaxes(title_text="각속도 (도/초)", row=3, col=1)
fig.update_yaxes(title_text="빈도", row=3, col=1)

fig.update_layout(title="방향 지표", height=1000, width=1000, showlegend=False)
fig.show()

위치 지표

In [ ]:
import numpy as np

mouse_id = ids[0]

unit_width = 400
unit_height = 400

# first plot
fig1 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)

for i, body_part in enumerate(body_parts):
    position = np.array(metrics[mouse_id][body_part]["position"])
    timestamp = metrics[mouse_id][body_part]["timestamp"]
    num_points = len(position)
    colors = np.linspace(0, 1, num_points - 1)
    fig1.add_trace(
        go.Scatter(
            x=position[:, 0],
            y=position[:, 1],
            mode="markers+lines",
            marker=dict(
                size=4,
                color=timestamp,
                colorscale="Viridis",
                colorbar=dict(title="시간 (초)") if i == 0 else None,
            ),
            line=dict(color="rgba(0,0,0,0.3)"),
        ),
        row=1,
        col=i + 1,
    )
    fig1.update_xaxes(range=[0, img_width], row=1, col=i + 1)
    fig1.update_yaxes(range=[0, img_height], row=1, col=i + 1)
fig1.update_layout(
    title="궤적",
    height=unit_height,
    width=unit_width * len(body_parts),
    showlegend=False,
)
fig1.show()

# second plot
fig2 = make_subplots(rows=1, cols=len(body_parts), subplot_titles=body_part_names)
for i, body_part in enumerate(body_parts):
    position = np.array(metrics[mouse_id][body_part]["position"])
    histgram_2d, x_edges, y_edges = np.histogram2d(
        position[:, 0],
        position[:, 1],
        bins=(50, 50),
        range=[[0, img_width], [0, img_height]],
    )
    go_2 = go.Heatmap(z=histgram_2d, x=x_edges, y=y_edges)
    go_2.colorbar.update(
        tickmode="array",
        tickvals=[histgram_2d.min(), histgram_2d.max()],
        ticktext=["낮음", "높음"],
    )
    fig2.add_trace(go_2, row=1, col=i + 1)
    fig2.update_xaxes(range=[0, img_width], row=1, col=i + 1)
    fig2.update_yaxes(range=[0, img_height], row=1, col=i + 1)

fig2.update_layout(
    title="히트맵",
    height=unit_height,
    width=unit_width * len(body_parts),
)
fig2.show()